# Data Resampling - Minute

In [1]:
import dai
import pandas as pd
from datetime import datetime, timedelta
from bigcharts import Kline

In [2]:
sd = '2026-02-13 00:00:00.000'
ed = '2026-02-13 23:59:59.000'

## 1 Minute

In [3]:
sql_01m = """ 
WITH 
data_base AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
    FROM cn_stock_bar1m
    WHERE instrument = '600519.SH'
)
SELECT * 
FROM data_base
"""

df_01m = dai.query(sql_01m, filters={"date":[sd, ed]}).df()
df_01m

,date,instrument,open,close,high,low,volume,amount
0,2026-02-13 09:25:00,600519.SH,1486.60,1486.60,1486.60,1486.60,32200,4.786852e+07
1,2026-02-13 09:31:00,600519.SH,1486.60,1483.00,1490.60,1483.00,62355,9.272921e+07
2,2026-02-13 09:32:00,600519.SH,1482.75,1473.44,1482.75,1473.00,60300,8.905925e+07
3,2026-02-13 09:33:00,600519.SH,1473.89,1474.00,1474.98,1471.81,77100,1.136032e+08
4,2026-02-13 09:34:00,600519.SH,1472.88,1471.52,1475.39,1471.52,56500,8.319221e+07
...,...,...,...,...,...,...,...,...
236,2026-02-13 14:56:00,600519.SH,1484.87,1484.60,1485.00,1484.58,18700,2.776633e+07
237,2026-02-13 14:57:00,600519.SH,1484.79,1485.50,1485.50,1484.59,27300,4.053695e+07
238,2026-02-13 14:58:00,600519.SH,1485.50,1485.50,1485.50,1485.50,1000,1.485401e+06
239,2026-02-13 14:59:00,600519.SH,1485.50,1485.50,1485.50,1485.50,0,0.000000e+00


In [4]:
Kline(df_01m)

## 5 Minute

In [5]:
sql_05m = """
WITH 
data_base AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
    FROM cn_stock_bar1m
    WHERE instrument = '600519.SH'
)
SELECT 
    date,
    instrument,
    DATE_TRUNC('hour', date) + INTERVAL '5 min' * CAST(FLOOR(EXTRACT(minute FROM date) / 5) AS BIGINT) AS _period,
    FIRST_VALUE(open) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
    LAST_VALUE(close) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
    MAX(high)         OVER (PARTITION BY _period, instrument) AS high,
    MIN(low)          OVER (PARTITION BY _period, instrument) AS low,
    SUM(amount)       OVER (PARTITION BY _period, instrument) AS amount,
    SUM(volume)       OVER (PARTITION BY _period, instrument) AS volume,
    row_number() OVER (PARTITION BY _period ORDER BY date) AS _rn,
FROM data_base
QUALIFY _rn = 1
ORDER BY date, instrument
"""

df_05m = dai.query(sql_05m, filters={"date":[sd, ed]}).df()
df_05m

,date,instrument,open,close,high,low,amount,volume
0,2026-02-13 09:25:00,600519.SH,1486.60,1486.60,1486.60,1486.60,4.786852e+07,32200
1,2026-02-13 09:31:00,600519.SH,1486.60,1471.52,1490.60,1471.52,3.785839e+08,256255
2,2026-02-13 09:35:00,600519.SH,1471.50,1482.01,1482.03,1470.58,2.851995e+08,193200
3,2026-02-13 09:40:00,600519.SH,1483.00,1485.02,1492.06,1483.00,2.853208e+08,191800
4,2026-02-13 09:45:00,600519.SH,1485.05,1487.57,1490.24,1485.05,1.441654e+08,96900
5,2026-02-13 09:50:00,600519.SH,1485.72,1490.08,1491.00,1485.56,1.538313e+08,103300
6,2026-02-13 09:55:00,600519.SH,1491.00,1492.08,1493.70,1490.30,2.676134e+08,179390
7,2026-02-13 10:00:00,600519.SH,1491.20,1498.01,1498.01,1491.20,1.895937e+08,126900
8,2026-02-13 10:05:00,600519.SH,1498.00,1499.99,1502.00,1496.51,2.689002e+08,179330
9,2026-02-13 10:10:00,600519.SH,1499.02,1499.02,1499.88,1496.50,9.647947e+07,64400


In [6]:
Kline(df_05m)

## 10 Minute

In [7]:
sql_10m = """
WITH 
data_base AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
    FROM cn_stock_bar1m
    WHERE instrument = '600519.SH'
)
SELECT 
    date,
    instrument,
    DATE_TRUNC('hour', date) + INTERVAL '10 min' * CAST(FLOOR(EXTRACT(minute FROM date) / 10) AS BIGINT) AS _period,
    FIRST_VALUE(open) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
    LAST_VALUE(close) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
    MAX(high)         OVER (PARTITION BY _period, instrument) AS high,
    MIN(low)          OVER (PARTITION BY _period, instrument) AS low,
    SUM(amount)       OVER (PARTITION BY _period, instrument) AS amount,
    SUM(volume)       OVER (PARTITION BY _period, instrument) AS volume,
    row_number() OVER (PARTITION BY _period ORDER BY date) AS _rn,
FROM data_base
QUALIFY _rn = 1
ORDER BY date, instrument
"""

df_10m = dai.query(sql_10m, filters={"date":[sd, ed]}).df()
df_10m

,date,instrument,open,close,high,low,amount,volume
0,2026-02-13 09:25:00,600519.SH,1486.60,1486.60,1486.60,1486.60,4.786852e+07,32200
1,2026-02-13 09:31:00,600519.SH,1486.60,1482.01,1490.60,1470.58,6.637834e+08,449455
2,2026-02-13 09:40:00,600519.SH,1483.00,1487.57,1492.06,1483.00,4.294863e+08,288700
3,2026-02-13 09:50:00,600519.SH,1485.72,1492.08,1493.70,1485.56,4.214447e+08,282690
4,2026-02-13 10:00:00,600519.SH,1491.20,1499.99,1502.00,1491.20,4.584939e+08,306230
5,2026-02-13 10:10:00,600519.SH,1499.02,1499.86,1499.88,1496.28,2.041816e+08,136300
6,2026-02-13 10:20:00,600519.SH,1499.86,1503.98,1505.50,1497.04,3.637271e+08,242100
7,2026-02-13 10:30:00,600519.SH,1503.98,1505.12,1507.80,1502.00,3.897956e+08,258924
8,2026-02-13 10:40:00,600519.SH,1504.71,1502.60,1506.00,1501.55,1.651217e+08,109770
9,2026-02-13 10:50:00,600519.SH,1502.37,1500.99,1503.43,1500.93,1.467920e+08,97725


In [8]:
Kline(df_10m)

## 15 Minute

In [9]:
sql_15m = """
WITH 
data_base AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
    FROM cn_stock_bar1m
    WHERE instrument = '600519.SH'
)
SELECT 
    date,
    instrument,
    DATE_TRUNC('hour', date) + INTERVAL '15 min' * CAST(FLOOR(EXTRACT(minute FROM date) / 15) AS BIGINT) AS _period,
    FIRST_VALUE(open) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
    LAST_VALUE(close) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
    MAX(high)         OVER (PARTITION BY _period, instrument) AS high,
    MIN(low)          OVER (PARTITION BY _period, instrument) AS low,
    SUM(amount)       OVER (PARTITION BY _period, instrument) AS amount,
    SUM(volume)       OVER (PARTITION BY _period, instrument) AS volume,
    row_number() OVER (PARTITION BY _period ORDER BY date) AS _rn,
FROM data_base
QUALIFY _rn = 1
ORDER BY date, instrument
"""

df_15m = dai.query(sql_15m, filters={"date":[sd, ed]}).df()
df_15m

,date,instrument,open,close,high,low,amount,volume
0,2026-02-13 09:25:00,600519.SH,1486.60,1486.60,1486.60,1486.60,4.786852e+07,32200
1,2026-02-13 09:31:00,600519.SH,1486.60,1485.02,1492.06,1470.58,9.491042e+08,641255
2,2026-02-13 09:45:00,600519.SH,1485.05,1492.08,1493.70,1485.05,5.656102e+08,379590
3,2026-02-13 10:00:00,600519.SH,1491.20,1499.02,1502.00,1491.20,5.549733e+08,370630
4,2026-02-13 10:15:00,600519.SH,1499.07,1503.98,1505.50,1496.28,4.714293e+08,314000
5,2026-02-13 10:30:00,600519.SH,1503.98,1504.20,1507.80,1502.00,4.856069e+08,322594
6,2026-02-13 10:45:00,600519.SH,1504.53,1500.99,1505.58,1500.93,2.161024e+08,143825
7,2026-02-13 11:00:00,600519.SH,1500.86,1499.00,1503.00,1498.20,1.867740e+08,124500
8,2026-02-13 11:15:00,600519.SH,1499.10,1493.01,1499.31,1492.49,2.509800e+08,167800
9,2026-02-13 11:30:00,600519.SH,1493.01,1493.01,1494.00,1492.53,1.075103e+07,7200


In [10]:
Kline(df_15m)

## 20 Minute

In [11]:
sql_20m = """
WITH 
data_base AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
    FROM cn_stock_bar1m
    WHERE instrument = '600519.SH'
),
data_periods AS (
    SELECT
        *,
        CASE 
            WHEN EXTRACT(minute FROM date) BETWEEN 11 AND 30 THEN DATE_TRUNC('hour', date) + INTERVAL '11 minute'
            WHEN EXTRACT(minute FROM date) BETWEEN 31 AND 50 THEN DATE_TRUNC('hour', date) + INTERVAL '31 minute'
            ELSE DATE_TRUNC('hour', date) + INTERVAL '51 minute'
        END AS period
    FROM data_base
)
SELECT 
    date,
    instrument,
    period AS _period,
    FIRST_VALUE(open) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
    LAST_VALUE(close) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
    MAX(high)         OVER (PARTITION BY _period, instrument) AS high,
    MIN(low)          OVER (PARTITION BY _period, instrument) AS low,
    SUM(amount)       OVER (PARTITION BY _period, instrument) AS amount,
    SUM(volume)       OVER (PARTITION BY _period, instrument) AS volume,
    row_number() OVER (PARTITION BY _period ORDER BY date) AS _rn,
FROM data_periods
QUALIFY _rn = 1
ORDER BY date, instrument
"""

df_20m = dai.query(sql_20m, filters={"date":[sd, ed]}).df()
df_20m

,date,instrument,open,close,high,low,amount,volume
0,2026-02-13 09:25:00,600519.SH,1486.60,1486.60,1486.60,1486.60,4.786852e+07,32200
1,2026-02-13 09:31:00,600519.SH,1486.60,1489.23,1492.06,1470.58,1.131039e+09,763555
2,2026-02-13 09:51:00,600519.SH,1489.23,1492.08,1493.70,1487.94,3.836751e+08,257290
3,2026-02-13 10:00:00,600519.SH,1491.20,1500.99,1503.43,1491.20,6.154597e+08,410755
4,2026-02-13 10:11:00,600519.SH,1497.99,1505.01,1505.50,1496.28,5.834733e+08,388700
5,2026-02-13 10:31:00,600519.SH,1504.82,1502.37,1507.80,1501.55,5.291789e+08,351594
6,2026-02-13 11:00:00,600519.SH,1500.86,1500.79,1503.00,1499.50,1.106131e+08,73700
7,2026-02-13 11:11:00,600519.SH,1502.00,1493.01,1502.00,1492.49,3.378919e+08,225800
8,2026-02-13 13:01:00,600519.SH,1493.01,1486.22,1505.01,1485.50,4.215810e+08,281739
9,2026-02-13 13:11:00,600519.SH,1500.19,1490.76,1501.13,1490.75,2.608392e+08,174500


In [12]:
Kline(df_20m)

## 30 Minute

In [13]:
sql_30m = """
WITH 
data_base AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount,
    FROM cn_stock_bar1m
    WHERE instrument = '600519.SH'
)
SELECT 
    date,
    instrument,
    DATE_TRUNC('hour', date) + INTERVAL '30 min' * CAST(FLOOR(EXTRACT(minute FROM date) / 30) AS BIGINT) AS _period,
    FIRST_VALUE(open) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open,
    LAST_VALUE(close) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
    MAX(high)         OVER (PARTITION BY _period, instrument) AS high,
    MIN(low)          OVER (PARTITION BY _period, instrument) AS low,
    SUM(amount)       OVER (PARTITION BY _period, instrument) AS amount,
    SUM(volume)       OVER (PARTITION BY _period, instrument) AS volume,
    row_number() OVER (PARTITION BY _period ORDER BY date) AS _rn,
FROM data_base
QUALIFY _rn = 1
ORDER BY date, instrument
"""

df_30m = dai.query(sql_30m, filters={"date":[sd, ed]}).df()
df_30m

,date,instrument,open,close,high,low,amount,volume
0,2026-02-13 09:25:00,600519.SH,1486.60,1486.60,1486.60,1486.60,4.786852e+07,32200
1,2026-02-13 09:31:00,600519.SH,1486.60,1492.08,1493.70,1470.58,1.514714e+09,1020845
2,2026-02-13 10:00:00,600519.SH,1491.20,1503.98,1505.50,1491.20,1.026403e+09,684630
3,2026-02-13 10:30:00,600519.SH,1503.98,1500.99,1507.80,1500.93,7.017093e+08,466419
4,2026-02-13 11:00:00,600519.SH,1500.86,1493.01,1503.00,1492.49,4.377540e+08,292300
5,2026-02-13 11:30:00,600519.SH,1493.01,1493.01,1494.00,1492.53,1.075103e+07,7200
6,2026-02-13 13:01:00,600519.SH,1493.01,1491.09,1505.01,1491.00,5.121150e+08,341800
7,2026-02-13 13:30:00,600519.SH,1491.10,1486.22,1491.36,1485.50,4.636207e+08,311358
8,2026-02-13 14:00:00,600519.SH,1486.01,1483.31,1488.00,1481.10,6.175416e+08,416061
9,2026-02-13 14:30:00,600519.SH,1483.85,1485.50,1489.80,1483.61,8.256026e+08,555837


In [14]:
Kline(df_30m)

## 1 Hour

In [15]:
sql_1h = """
WITH 
data_base AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount
    FROM cn_stock_bar1m
    WHERE instrument = '600519.SH'
),
data_periods AS (
    SELECT
        *,
        CASE 
            WHEN date::time >= TIME '09:25:00' AND date::time <= TIME '10:30:00' THEN DATE_TRUNC('day', date) + INTERVAL '09:25:00'
            WHEN date::time >= TIME '10:30:00' AND date::time <= TIME '11:30:00' THEN DATE_TRUNC('day', date) + INTERVAL '10:30:00'
            WHEN date::time >= TIME '13:01:00' AND date::time <= TIME '14:00:00' THEN DATE_TRUNC('day', date) + INTERVAL '13:01:00'
            WHEN date::time >= TIME '14:00:00' AND date::time <= TIME '15:00:00' THEN DATE_TRUNC('day', date) + INTERVAL '14:00:00'
        END AS period
    FROM data_base
    WHERE date::time >= TIME '09:25:00' AND date::time <= TIME '15:00:00'
)
SELECT 
    date,
    instrument,
    period AS _period,
    FIRST_VALUE(open) OVER (PARTITION BY _period, instrument ORDER BY date) AS open,
    LAST_VALUE(close) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
    MAX(high)         OVER (PARTITION BY _period, instrument) AS high,
    MIN(low)          OVER (PARTITION BY _period, instrument) AS low,
    SUM(amount)       OVER (PARTITION BY _period, instrument) AS amount,
    SUM(volume)       OVER (PARTITION BY _period, instrument) AS volume,
    ROW_NUMBER()      OVER (PARTITION BY _period ORDER BY date) AS _rn
FROM data_periods
QUALIFY _rn = 1
ORDER BY _period, instrument
"""

df_1h = dai.query(sql_1h, filters={"date":[sd, ed]}).df()
df_1h

,date,instrument,open,close,high,low,amount,volume
0,2026-02-13 09:25:00,600519.SH,1486.60,1505.01,1505.50,1470.58,2.621335e+09,1759175
1,2026-02-13 10:31:00,600519.SH,1504.82,1493.01,1507.80,1492.49,1.117865e+09,744419
2,2026-02-13 13:01:00,600519.SH,1493.01,1487.99,1505.01,1485.50,9.877773e+08,661258
3,2026-02-13 14:01:00,600519.SH,1486.99,1485.30,1489.80,1481.10,1.489402e+09,1003049


In [16]:
Kline(df_1h)

## 2 Hour

In [17]:
sql_2h = """
WITH 
data_base AS (
    SELECT
        date,
        instrument,
        open,
        close,
        high,
        low,
        volume,
        amount
    FROM cn_stock_bar1m
    WHERE instrument = '600519.SH'
),
data_periods AS (
    SELECT
        *,
        CASE 
            WHEN date::time >= TIME '09:25:00' AND date::time <= TIME '11:30:00' THEN DATE_TRUNC('day', date) + INTERVAL '09:25:00'
            WHEN date::time >= TIME '13:01:00' AND date::time <= TIME '15:00:00' THEN DATE_TRUNC('day', date) + INTERVAL '13:01:00'
        END AS period
    FROM data_base
    WHERE date::time >= TIME '09:25:00' AND date::time <= TIME '15:00:00'
)
SELECT 
    date,
    instrument,
    period AS _period,
    FIRST_VALUE(open) OVER (PARTITION BY _period, instrument ORDER BY date) AS open,
    LAST_VALUE(close) OVER (PARTITION BY _period, instrument ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close,
    MAX(high)         OVER (PARTITION BY _period, instrument) AS high,
    MIN(low)          OVER (PARTITION BY _period, instrument) AS low,
    SUM(amount)       OVER (PARTITION BY _period, instrument) AS amount,
    SUM(volume)       OVER (PARTITION BY _period, instrument) AS volume,
    ROW_NUMBER()      OVER (PARTITION BY _period ORDER BY date) AS _rn
FROM data_periods
QUALIFY _rn = 1
ORDER BY _period, instrument
"""

df_2h = dai.query(sql_2h, filters={"date":[sd, ed]}).df()
df_2h

,date,instrument,open,close,high,low,amount,volume
0,2026-02-13 09:25:00,600519.SH,1486.60,1493.01,1507.80,1470.58,3.739200e+09,2503594
1,2026-02-13 13:01:00,600519.SH,1493.01,1485.30,1505.01,1481.10,2.477179e+09,1664307


In [18]:
Kline(df_2h)